<a href="https://colab.research.google.com/github/riddhi-mehta-1721/NLP_Project/blob/master/Book_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP PROJECT

## RIDDHI MEHTA - J030
## HUSAIN GHADIALI - J056

# Importing Libraries

In [1]:
import pandas as pd
import numpy as np

import re

import seaborn as sns
import matplotlib.pyplot as plt

import pydotplus
import pydot

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


from tensorflow.python.keras.layers import Dense, Embedding, Input, LSTM, Bidirectional, GlobalMaxPool1D, Dropout, Conv1D, MaxPooling1D
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
# from keras.models import Model
# from keras.layers import Dense, Embedding, Input, LSTM, Bidirectional, GlobalMaxPool1D, Dropout
# from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import plot_model  
from keras.utils.vis_utils import model_to_dot

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

import warnings
warnings.filterwarnings("ignore")

import random

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Mounting Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Reading Data

In [0]:
data = pd.read_csv(r"/content/drive/My Drive/Colab Notebooks/NLP - Sem 6/Project/Datasets/original.csv")

In [4]:
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [0]:
data.drop(columns = "id", inplace = True)

In [0]:
data.dropna(inplace = True)

In [7]:
data.shape

(159571, 7)

# Data Pre Processing

In [8]:
data["toxic"].sum(),data["severe_toxic"].sum(),data["obscene"].sum(),data["threat"].sum(),data["insult"].sum(),data["identity_hate"].sum()

(15294, 1595, 8449, 478, 7877, 1405)

In [9]:
print("Number of total sentences:  ", end = "")
print(data.shape[0])
print("Number of positive sentences:  ", end = "")
print(data[(data.toxic == 0) & (data.insult == 0) & (data.severe_toxic == 0) & (data.obscene == 0) & (data.threat == 0) & (data.identity_hate == 0)].shape[0])
print("Number of sentences with one or more labels:  ", end = "")
print(data.shape[0] - data[(data.toxic == 0) & (data.insult == 0) & (data.severe_toxic == 0) & (data.obscene == 0) & (data.threat == 0) & (data.identity_hate == 0)].shape[0])

Number of total sentences:  159571
Number of positive sentences:  143346
Number of sentences with one or more labels:  16225


In [0]:
def preprocessing(text):
  
  text = text.lower()
  text = re.sub(r"what's", "what is ", text)
  text = re.sub(r"\'s", " ", text)
  text = re.sub(r"\'ve", " have ", text)
  text = re.sub(r"can't", "cannot ", text)
  text = re.sub(r"n't", " not ", text)
  text = re.sub(r"i'm", "i am ", text)
  text = re.sub(r"\'re", " are ", text)
  text = re.sub(r"\'d", " would ", text)
  text = re.sub(r"\'ll", " will ", text)
  text = re.sub(r"\'scuse", " excuse ", text)
  text = re.sub('\W', ' ', text) # substitutes non word character with a space
  text = re.sub('\s+', ' ', text)
  text = text.strip(' ')
  text = re.sub(r"[0-9]"," ",text) # substitutes numbers with a blank space

  my_tokens = list() # Initialising a list of tokens that will be formed
  stop_words= set(stopwords.words('english')) # Extracting english stopwords from nltk corpus

  tokens = word_tokenize(text, preserve_line=False)

  for token in tokens:

    if (token not in stop_words):
      my_tokens.append(token)
  
  final = ' '.join([str(elem) for elem in my_tokens]) # makes it an entire string

  return final

In [0]:
data["comment_text"] = data["comment_text"].apply(preprocessing)

# Separating Positive and Negative Sentences

In [0]:
positive_sentences = data[(data.toxic == 0) & (data.insult == 0) & (data.severe_toxic == 0) & (data.obscene == 0) & (data.threat == 0) & (data.identity_hate == 0)]
negative_sentences = data.drop((data[(data.toxic == 0) & (data.insult == 0) & (data.severe_toxic == 0) & (data.obscene == 0) & (data.threat == 0) & (data.identity_hate == 0)].index))
positive_sentences.shape, negative_sentences.shape

((143346, 7), (16225, 7))

In [0]:
positive_sentences.reset_index(inplace = True)
negative_sentences.reset_index(inplace = True)

positive_sentences.drop(columns = "index", inplace = True)
negative_sentences.drop(columns = "index", inplace = True)

# Outlier Cleaning

### 1. Calculating length of each sentence

In [0]:
def length_of_sentence(text):
  return len(word_tokenize(text))

In [0]:
positive_sentences["length"] = positive_sentences["comment_text"].apply(length_of_sentence)
negative_sentences["length"] = negative_sentences["comment_text"].apply(length_of_sentence)

In [0]:
positive_sentences.head(3)

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,length
0,explanation edits made username hardcore metal...,0,0,0,0,0,0,23
1,aww matches background colour seemingly stuck ...,0,0,0,0,0,0,10
2,hey man really trying edit war guy constantly ...,0,0,0,0,0,0,21


In [0]:
negative_sentences.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,length
0,cocksucker piss around work,1,1,1,0,1,0,4
1,hey talk exclusive group wp talibans good dest...,1,0,0,0,0,0,29
2,bye look come think comming back tosser,1,0,0,0,0,0,7
3,gay antisemmitian archangel white tiger meow g...,1,0,1,0,1,1,62
4,fuck filthy mother ass dry,1,0,1,0,1,0,5


### 2. Removing Null Values in both the Dataframes

In [0]:
positive_sentences[positive_sentences["length"] == 0].shape[0], negative_sentences[negative_sentences["length"] == 0].shape[0]

(58, 0)

In [0]:
positive_sentences[positive_sentences["length"] == 0].index

Int64Index([  1877,   2150,   3564,   4003,   5624,   7921,   8412,  11166,
             15561,  20719,  21094,  22964,  23069,  23692,  27837,  28700,
             34332,  37330,  38795,  42196,  47049,  48248,  52580,  55428,
             57706,  62978,  64371,  67070,  71580,  73801,  74208,  76557,
             84284,  86943,  90524,  91320,  93392,  98404,  99238, 102553,
            102958, 107719, 111784, 111930, 113026, 114023, 115499, 115926,
            126199, 128349, 128558, 131301, 132905, 133756, 135994, 137270,
            137726, 142848],
           dtype='int64')

In [0]:
positive_sentences.drop((positive_sentences[positive_sentences["length"] == 0].index), inplace = True)

In [0]:
positive_sentences.reset_index(inplace = True)
positive_sentences.drop(columns = "index", inplace = True)

In [0]:
positive_sentences[positive_sentences["length"] == 0].shape[0], negative_sentences[negative_sentences["length"] == 0].shape[0]

(0, 0)

### 3. Dropping rows where length of each sentence is beyond the range 25% - 75% percentile

#### 3a. For "positive_sentences"

In [0]:
positive_sentences["length"].describe()

count    143288.000000
mean         35.405009
std          52.178395
min           1.000000
25%           9.000000
50%          19.000000
75%          39.000000
max        1250.000000
Name: length, dtype: float64

In [0]:
positive_sentences["length"].describe()[4], positive_sentences["length"].describe()[6]

(9.0, 39.0)

In [0]:
positive_sentences[(positive_sentences.length >= positive_sentences["length"].describe()[4]) &
                   (positive_sentences.length <= positive_sentences["length"].describe()[6])].shape

(74136, 8)

In [0]:
positive_sentences = positive_sentences[(positive_sentences.length >= positive_sentences["length"].describe()[4]) &
                   (positive_sentences.length <= positive_sentences["length"].describe()[6])]

In [0]:
positive_sentences["length"].describe()

count    74136.000000
mean        20.198689
std          8.460624
min          9.000000
25%         13.000000
50%         19.000000
75%         26.000000
max         39.000000
Name: length, dtype: float64

In [0]:
positive_sentences.drop(columns = "length", inplace = True)

In [0]:
positive_sentences.reset_index(inplace = True)
positive_sentences.drop(columns = "index", inplace = True)

#### 3b. For "negative_sentences"

In [0]:
negative_sentences["length"].describe()

count    16225.000000
mean        28.746872
std         68.379872
min          1.000000
25%          6.000000
50%         12.000000
75%         25.000000
max       1250.000000
Name: length, dtype: float64

In [0]:
negative_sentences = negative_sentences[(negative_sentences.length >= negative_sentences["length"].describe()[4]) &
                   (negative_sentences.length <= negative_sentences["length"].describe()[6])]

In [0]:
negative_sentences["length"].describe()

count    8688.000000
mean       12.788329
std         5.484364
min         6.000000
25%         8.000000
50%        12.000000
75%        17.000000
max        25.000000
Name: length, dtype: float64

In [0]:
negative_sentences.drop(columns = "length", inplace = True)

In [0]:
negative_sentences.reset_index(inplace = True)
negative_sentences.drop(columns = "index", inplace = True)

In [0]:
print("TOXIC:  ", end = "")
print(negative_sentences["toxic"].sum())

print("SEVERE_TOXIC:  ", end = "")
print(negative_sentences["severe_toxic"].sum())

print("INSULT:  ", end = "")
print(negative_sentences["insult"].sum())

print("THREAT:  ", end = "")
print(negative_sentences["threat"].sum())

print("IDENTITY_HATE:  ", end = "")
print(negative_sentences["identity_hate"].sum())

print("OBSCENE:  ", end = "")
print(negative_sentences["obscene"].sum())

TOXIC:  8266
SEVERE_TOXIC:  848
INSULT:  4319
THREAT:  285
IDENTITY_HATE:  780
OBSCENE:  4487


From the above data it is clear that, it will be difficult to identify the following labels:

1. Threat
2. Identity_Hate
3. Severe_Toxic

# Adding 2 more columns: "positive" and "negative" which will be binary labels to classify a sentence into these two categories

In [0]:
labels = ["positive", "negative"]
d = dict.fromkeys(labels, 0)

In [0]:
positive_sentences = positive_sentences.assign(**d)
negative_sentences = negative_sentences.assign(**d)

In [0]:
positive_sentences.positive = positive_sentences.positive.replace(0,1)

In [0]:
negative_sentences.negative = negative_sentences.negative.replace(0,1)

In [0]:
positive_sentences.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,positive,negative
0,explanation edits made username hardcore metal...,0,0,0,0,0,0,1,0
1,aww matches background colour seemingly stuck ...,0,0,0,0,0,0,1,0
2,hey man really trying edit war guy constantly ...,0,0,0,0,0,0,1,0
3,sorry word nonsense offensive anyway intending...,0,0,0,0,0,0,1,0
4,oh girl started arguments stuck nose belong be...,0,0,0,0,0,0,1,0


In [0]:
negative_sentences.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,positive,negative
0,bye look come think comming back tosser,1,0,0,0,0,0,0,1
1,sorry sorry screwed around someones talk page ...,1,0,0,0,0,0,0,1
2,get fucked get fuckeeed got drink cant put get...,1,0,1,0,0,0,0,1
3,stupid peace shit stop deleting stuff asshole ...,1,1,1,0,1,0,0,1
4,tony sidaway obviously fistfuckee loves arm ass,1,0,1,0,1,0,0,1


# Saving these DataFrames for direct use in future

In [0]:
positive_sentences.to_csv(r"/content/drive/My Drive/Colab Notebooks/NLP - Sem 6/Project/Datasets/positive_sentences.csv")
negative_sentences.to_csv(r"/content/drive/My Drive/Colab Notebooks/NLP - Sem 6/Project/Datasets/negative_sentences.csv")

# ------------------ THE END ------------------